# Initilization

In [1]:
!mkdir -p models
!wget "https://drive.google.com/uc?id=1E95HNEYQI1R-UTuYwJOycrYDJxFxiICC&export=download" -O songs.p

--2020-04-03 04:35:47--  https://drive.google.com/uc?id=1E95HNEYQI1R-UTuYwJOycrYDJxFxiICC&export=download
Resolving drive.google.com (drive.google.com)... 64.233.189.101, 64.233.189.138, 64.233.189.113, ...
Connecting to drive.google.com (drive.google.com)|64.233.189.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ot62h577od63001q9c1k4omk7bp23t03/1585888500000/14610559106662224301/*/1E95HNEYQI1R-UTuYwJOycrYDJxFxiICC?e=download [following]
--2020-04-03 04:35:47--  https://doc-08-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ot62h577od63001q9c1k4omk7bp23t03/1585888500000/14610559106662224301/*/1E95HNEYQI1R-UTuYwJOycrYDJxFxiICC?e=download
Resolving doc-08-3s-docs.googleusercontent.com (doc-08-3s-docs.googleusercontent.com)... 74.125.203.132, 2404:6800:4008:c03::84
Connecting to doc-08-3s-docs.googleusercontent.com (doc-08

# Dataset Preparation

In [2]:
%pylab inline
import pickle
import pandas as pd
import keras
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential, load_model
from keras.layers import LSTM, Bidirectional, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint, History

# implementation based on https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5
def generate_dataset():
  """datas preprocessing based on a list of song objects, each song object contains song's name and a sequence of notes"""
  songs = pickle.load(open("songs.p", "rb"))

  # get a list of all notes in all songs
  notes = []
  for song in songs:
    notes += song["notes"]

  # n_vocab is the number of unique netes
  n_vocab = len(set(notes))

  # length of input sequence to LSTM network 
  sequence_length = 100
  # get all pitch names
  pitchnames = sorted(set(item for item in notes))
  # create a dictionary to map pitches to integers
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
  int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

  network_input = []
  network_output = []
  # create input sequences and the corresponding outputs
  for song in songs:
    print("Loading", song["name"])
    notes = song["notes"]

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

  n_patterns = len(network_input)
  # reshape the input into a format compatible with LSTM layers
  network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
  # normalize input
  network_input = network_input / float(n_vocab)
  network_output = keras.utils.to_categorical(network_output)

  return (n_vocab, int_to_note, network_input, network_output)

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

# Create Network

In [0]:
def create_network(network_input, n_vocab):
    """create network structure"""
    model = Sequential()
    model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(512, return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(512)))
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

# Train Network

In [0]:
def train_network():
  """train the network"""
  (n_vocab, _, network_input, network_output) = generate_dataset()

  # get model structure
  model = create_network(network_input, n_vocab)
  model.summary()

  # callbacks
  history = History()
  filepath = 'models/model-{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5'
  checkpoint_cb = ModelCheckpoint(
      filepath, monitor='loss', verbose=0, save_best_only=True, mode='min'
  )

  # start training the model
  n_epoch = 20
  model.fit(network_input, network_output, epochs=n_epoch, batch_size=64, validation_split=0.2,
            callbacks=[history, checkpoint_cb])
  model.save('music_generate_model.h5')

  # Plot the model losses
  pd.DataFrame(history.history).plot()
  plt.savefig('network_loss_per_epoch.png', transparent=True)
  plt.close()

# Model Prediction

In [8]:
# Model predicion's implementation based on https://github.com/corynguyen19/midi-lstm-gan

def generate_notes(model):
    """ Generate notes from the neural network based on a sequence of notes """
    (n_vocab, int_to_note, network_input, _) = generate_dataset()

    # pick a random sequence from the input as a starting point for the prediction  
    start = np.random.randint(0, len(network_input) - 1)

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for i in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        # random choose from a distribution
        index = np.random.choice(n_vocab, 1, p=prediction[0])[0]
        # index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        print(result)
        
        pattern = np.append(pattern, index)
        pattern = pattern[1 : len(pattern)]

    return prediction_output
  

def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))

model = load_model('/content/models/model-20-1.53-6.50.hdf5')
prediction_output = generate_notes(model)
create_midi(prediction_output, 'test')

Loading I Will (From \"Ao Haru Ride\")
Loading Sword Art Online A Tender Feeling
Loading Sword Art Online At Our Parting
Loading New Empire - A Little Braver
Loading La Corda d'Oro Tsumugareru Kioku
Loading VIXX - Error
Loading Euterpe MIDI
Loading Diabolik Lovers - Yui
Loading Ignite - Sword Art Online
Loading Aqua Terrarium 
Loading UtaPri Heavens Gate (Orchestra)
Loading UtaPri Sanctuary
Loading Nana - A little pain-1
Loading Goblin - Beautiful (AnimeMidi)
Loading NieR - Weight of the World (Solo)
Loading Vogel im Ka_fig
Loading Diabolik Lovers - Rosary
Loading UtaPri Rainbow Dream
Loading Senjougahara Tore @
Loading Catch the Moment SAO
Loading A Werewolf Boy - A Werewolf Boy
Loading Brothers Conflict BelovedXSurvival
Loading Angel Beats! - Ichiban no Takaramono 2017
Loading Hiiro no Kakera - Nee (AnimeMidi)
Loading Tetsukazu no Kanjou (Piano Solo)
Loading Dango Daikazoku
Loading BTS - Go Go (AnimeMidi)
Loading Final Fantasy XV - Valse di Fantasica
Loading Diabolik Lovers - More Bl

#Download files

In [3]:
from google.colab import files
from google.colab import drive

# files.download('test.mid')
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!zip models.zip models/*
!cp models.zip '/content/gdrive/My Drive/'

	zip warning: name not matched: models/*

zip error: Nothing to do! (models.zip)
cp: cannot stat 'models.zip': No such file or directory


In [4]:
!unzip /content/gdrive/My\ Drive/models.zip

Archive:  /content/gdrive/My Drive/models.zip
   creating: models/
  inflating: models/model-01-4.68-4.62.hdf5  
  inflating: models/model-02-4.56-4.59.hdf5  
  inflating: models/model-03-4.51-4.54.hdf5  
  inflating: models/model-04-4.42-4.51.hdf5  
  inflating: models/model-05-4.35-4.51.hdf5  
  inflating: models/model-06-4.28-4.55.hdf5  
  inflating: models/model-07-4.21-4.51.hdf5  
  inflating: models/model-08-4.10-4.55.hdf5  
  inflating: models/model-09-3.97-4.67.hdf5  
  inflating: models/model-10-3.84-4.68.hdf5  
  inflating: models/model-11-3.65-4.77.hdf5  
  inflating: models/model-12-3.47-4.72.hdf5  
  inflating: models/model-13-3.26-4.91.hdf5  
  inflating: models/model-14-3.05-4.88.hdf5  
  inflating: models/model-15-2.82-5.22.hdf5  
  inflating: models/model-16-2.58-5.50.hdf5  
  inflating: models/model-17-2.31-5.47.hdf5  
  inflating: models/model-18-2.05-5.94.hdf5  
  inflating: models/model-19-1.79-6.20.hdf5  
  inflating: models/model-20-1.53-6.50.hdf5  


In [10]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 27kB/s 
     |████████████████████████████████| 491kB 41.5MB/s 
     |████████████████████████████████| 3.2MB 49.6MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2
